In [1]:
import pandas as pd

df = pd.read_parquet('c:/A1/new_subway1.parquet')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58799739 entries, 0 to 58799738
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   transit_timestamp   object 
 1   station_complex_id  object 
 2   station_complex     object 
 3   borough             object 
 4   ridership           int16  
 5   latitude            float32
 6   longitude           float32
dtypes: float32(2), int16(1), object(4)
memory usage: 2.3+ GB


,transit_timestamp,station_complex_id,station_complex,borough,ridership,latitude,longitude
0,06/09/2023 12:00:00 AM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,4,40.632835,-73.947639
1,05/03/2023 12:00:00 PM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,58,40.632835,-73.947639
2,08/20/2023 03:00:00 PM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,12,40.632835,-73.947639
3,06/06/2023 11:00:00 PM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,1,40.632835,-73.947639
4,08/10/2023 11:00:00 AM,359,"Flatbush Av-Brooklyn College (2,5)",Brooklyn,14,40.632835,-73.947639


In [2]:
# Convert object types to category
for col in df.select_dtypes(include=['object']).columns:
    if col!='transit_timestamp':
        df[col] = df[col].astype('category')

# Check the data types to confirm changes
print(df.dtypes)

transit_timestamp       object
station_complex_id    category
station_complex       category
borough               category
ridership                int16
latitude               float32
longitude              float32
dtype: object


In [3]:
df['transit_timestamp'] = df['transit_timestamp'].astype('category')
df.drop(columns=['borough', 'station_complex'], inplace=True)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58799739 entries, 0 to 58799738
Data columns (total 5 columns):
 #   Column              Dtype   
---  ------              -----   
 0   transit_timestamp   category
 1   station_complex_id  category
 2   ridership           int16   
 3   latitude            float32 
 4   longitude           float32 
dtypes: category(2), float32(2), int16(1)
memory usage: 785.7 MB


In [5]:
print(len(df))

58799739


In [4]:
df.head()
print(df['transit_timestamp'].value_counts())

transit_timestamp
05/29/2024 08:00:00 AM    3705
05/15/2024 08:00:00 AM    3704
03/20/2024 08:00:00 AM    3696
05/28/2024 08:00:00 AM    3696
05/01/2024 08:00:00 AM    3695
                          ... 
12/31/2022 11:00:00 PM       9
12/31/2022 01:00:00 AM       8
12/31/2022 07:00:00 AM       8
12/31/2022 05:00:00 AM       4
12/31/2022 02:00:00 AM       2
Name: count, Length: 20395, dtype: int64


In [5]:
df.tail()

,transit_timestamp,station_complex_id,ridership,latitude,longitude
58799734,05/13/2023 04:00:00 PM,359,3,40.632835,-73.947639
58799735,03/27/2023 03:00:00 AM,359,1,40.632835,-73.947639
58799736,01/27/2023 01:00:00 AM,359,1,40.632835,-73.947639
58799737,05/29/2023 07:00:00 PM,359,7,40.632835,-73.947639
58799738,01/17/2023 06:00:00 AM,359,65,40.632835,-73.947639


In [6]:
df.drop(columns=['latitude', 'longitude'], inplace=True)

In [7]:
new = df.groupby(['transit_timestamp', 'station_complex_id'], as_index=False)['ridership'].sum()

C:\Users\Kevin\AppData\Local\Temp\ipykernel_15544\2793718700.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  new = df.groupby(['transit_timestamp', 'station_complex_id'], as_index=False)['ridership'].sum()


In [8]:
df.head(
)
df.info()
print(len(new))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58799739 entries, 0 to 58799738
Data columns (total 3 columns):
 #   Column              Dtype   
---  ------              -----   
 0   transit_timestamp   category
 1   station_complex_id  category
 2   ridership           int16   
dtypes: category(2), int16(1)
memory usage: 337.1 MB
8729060


In [9]:
new.head()

,transit_timestamp,station_complex_id,ridership
0,01/01/2023 01:00:00 AM,1,52
1,01/01/2023 01:00:00 AM,10,95
2,01/01/2023 01:00:00 AM,100,52
3,01/01/2023 01:00:00 AM,101,249
4,01/01/2023 01:00:00 AM,103,394


In [1]:
import pandas as pd
df = pd.read_parquet('c:/A1/ready_subway.parquet')

In [10]:
new.tail()

,transit_timestamp,station_complex_id,ridership
8729055,12/31/2023 12:00:00 PM,97,304
8729056,12/31/2023 12:00:00 PM,98,127
8729057,12/31/2023 12:00:00 PM,99,92
8729058,12/31/2023 12:00:00 PM,TRAM1,536
8729059,12/31/2023 12:00:00 PM,TRAM2,477


In [17]:
subway = pd.read_csv('c:/A3/MTA_Subway_Stations.csv', usecols=['Complex ID', 'GTFS Longitude', 'GTFS Latitude'])
subway = subway.rename(columns={
    'Complex ID': 'station_complex_id',
    'GTFS Longitude': 'longitude',
    'GTFS Latitude': 'latitude'
})
subway['station_complex_id'] = subway['station_complex_id'].astype(str)

In [34]:
new.nunique()

transit_timestamp     20395
station_complex_id      426
ridership             11800
latitude                473
longitude               473
dtype: int64

In [18]:
new = new.merge(subway, on='station_complex_id', how='left')

,transit_timestamp,station_complex_id,ridership
0,06/09/2023 12:00:00 AM,359,4
1,05/03/2023 12:00:00 PM,359,58
2,08/20/2023 03:00:00 PM,359,12
3,06/06/2023 11:00:00 PM,359,1
4,08/10/2023 11:00:00 AM,359,14


In [29]:

print(new.tail(30))

              transit_timestamp station_complex_id  ridership   latitude  \
9769175  12/31/2023 12:00:00 PM                 72        188  40.631386   
9769176  12/31/2023 12:00:00 PM                 74        161  40.620671   
9769177  12/31/2023 12:00:00 PM                 75         84  40.617410   
9769178  12/31/2023 12:00:00 PM                 76        217  40.611815   
9769179  12/31/2023 12:00:00 PM                 77        125  40.603923   
9769180  12/31/2023 12:00:00 PM                 78        124  40.597473   
9769181  12/31/2023 12:00:00 PM                 79         49  40.592721   
9769182  12/31/2023 12:00:00 PM                  8        506  40.764811   
9769183  12/31/2023 12:00:00 PM                 80         30  40.700492   
9769184  12/31/2023 12:00:00 PM                 81         51  40.697418   
9769185  12/31/2023 12:00:00 PM                 82         59  40.695178   
9769186  12/31/2023 12:00:00 PM                 83         44  40.693879   
9769187  12/

In [36]:
new.drop(columns=['station_complex_id'], inplace=True)

In [37]:
datetime_format = '%m/%d/%Y %I:%M:%S %p'
new['transit_timestamp'] = pd.to_datetime(new['transit_timestamp'], format=datetime_format)

In [40]:
new['transit_timestamp'] = pd.to_datetime(new['transit_timestamp'])

In [45]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9728415 entries, 0 to 9769202
Data columns (total 4 columns):
 #   Column             Dtype   
---  ------             -----   
 0   transit_timestamp  category
 1   ridership          int16   
 2   latitude           float64 
 3   longitude          float64 
dtypes: category(1), float64(2), int16(1)
memory usage: 260.4 MB


In [47]:
new.head()

,transit_timestamp,ridership,latitude,longitude
0,2023-01-01 01:00:00,52,40.775036,-73.912034
1,2023-01-01 01:00:00,95,40.759901,-73.984139
2,2023-01-01 01:00:00,52,40.706870,-73.953431
3,2023-01-01 01:00:00,249,40.708359,-73.957757
4,2023-01-01 01:00:00,394,40.720280,-73.993915


In [48]:
new.to_parquet('c:/fomo_data/subway_final3.parquet')

In [30]:
new = new.dropna()
print(len(new))

9728415


In [10]:
import geopandas as gpd
from shapely.geometry import Point

# Load the location coordinates file as GeoDataFrame
location_coords_gdf = gpd.read_file('c:/A3/subway.shp')
print(location_coords_gdf.head())
subway = location_coords_gdf[['', 'longitude', 'latitude']]

grouped_with_coords = pd.merge(df, subway, on='station_complex_id', how='left')

grouped_with_coords['geometry'] = grouped_with_coords.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
df = gpd.GeoDataFrame(grouped_with_coords, geometry='geometry')
df.set_crs(epsg=4326, inplace=True)


nta_gdf = gpd.read_file('c:/A/nyct2020_24b/nyct2020_24b/nyct2020.shp')

# Perform spatial join to find the corresponding NTA for each pickup location
result_gdf = gpd.sjoin(df, nta_gdf, how='left', op='intersects')

# Now your result_gdf contains the NTA information
print(result_gdf.head())

          id name  objectid rt_symbol    shape_len  \
0  2000393.0    G     753.0         G  2438.200249   
1  2000394.0    G     754.0         G  3872.834411   
2  2000469.0    Q     755.0         N  1843.366331   
3  2000294.0    M     756.0         B  1919.559203   
4  2000296.0    M     757.0         B  2385.698536   

                                 url  \
0  http://web.mta.info/nyct/service/   
1  http://web.mta.info/nyct/service/   
2  http://web.mta.info/nyct/service/   
3  http://web.mta.info/nyct/service/   
4  http://web.mta.info/nyct/service/   

                                            geometry  
0  LINESTRING (-73.99488 40.68020, -73.99427 40.6...  
1  LINESTRING (-73.97958 40.65993, -73.97966 40.6...  
2  LINESTRING (-73.97586 40.57597, -73.97654 40.5...  
3  LINESTRING (-73.92414 40.75229, -73.92405 40.7...  
4  LINESTRING (-73.91345 40.75617, -73.90905 40.7...  


KeyError: "None of [Index(['', 'longitude', 'latitude'], dtype='object')] are in the [columns]"